In [1]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def save(np_img, name):
    np_img = np_img.astype(np.uint8)
    image = Image.fromarray(np_img)
    image.save(name)

In [3]:
#Thay đổi độ sáng
def change_brightness(np_img, val):
    img_brightness = np_img.copy()
    img_brightness = img_brightness + val
    img_brightness[img_brightness > 255] = 255
    
    return img_brightness


In [4]:
#thay đổi độ tương phản
def change_contrast(np_img, val):
    img_contrast = np_img.copy()
    
    factor = (259 * (val + 255)) / (255 * (259 - val))
    
    R = factor * (img_contrast[:, :, 0] - 128) + 128
    G = factor * (img_contrast[:, :, 1] - 128) + 128
    B = factor * (img_contrast[:, :, 2] - 128) + 128
    
    img_contrast[:, :, 0] = R
    img_contrast[:, :, 1] = G
    img_contrast[:, :, 2] = B
    
    img_contrast = img_contrast.astype(np.int64)
    img_contrast = np.clip(img_contrast,0,255)
    return img_contrast


In [5]:
#Lật ảnh theo chiều ngang.
def flip_horizontally(np_img):
    img_flip = np_img.copy()
    res = np.zeros(img_flip.shape, dtype='int64')
    size = len(img_flip[0]) - 1
    for i in range(size):
        res[:, i, :] = img_flip[:, size - i, :]
        
    return res

In [6]:
#Lật ảnh theo chiều dọc
def flip_vertically(np_img):
    img_flip = np_img.copy()
    res = np.zeros(img_flip.shape, dtype='int64')
    size = len(img_flip)-1
    for i in range(size):
        res[i, :, :] = img_flip[size - i, :, :]
    
    return res

In [7]:
#Chuyển thành ảnh xám
def gray_image(np_img):
    img_cop = np_img[:,:,:3] @ [0.2989, 0.5870, 0.1140]
    return img_cop


In [8]:
#Chồng hai ảnh có cùng kích thước
def blending_image(np_img, np_img2):
    img1_lenX = np_img.shape[0]
    img1_lenY = np_img.shape[1]
    img2_lenX = np_img2.shape[0]
    img2_lenY = np_img2.shape[1]
    if(img1_lenX != img2_lenX or img1_lenY != img2_lenY):
        return []
    brightness_1 = np.average(np_img)
    brightness_2 = np.average(np_img2)
    
    if brightness_1 > 50: # kiểm tra độ sáng của ảnh
        np_img = change_brightness(np_img,int(50-brightness_1))
    if brightness_2 > 50:
        np_img = change_brightness(np_img,int(50-brightness_2))
        
    img1_gray = gray_image(np_img)
    img2_gray = gray_image(np_img2)
    res = img1_gray + img2_gray
    res = np.clip(res, 0, 255)
    return res


In [9]:
#Làm mờ ảnh
def gaussian_blur(np_img):
    kernel = np.array([[1.0,2.0,1.0], [2.0,4.0,2.0], [1.0,2.0,1.0]])
    kernel = kernel / np.sum(kernel)
    array = []
    
    for i in range(3):
        img_array = np.copy(np_img)
        img_array = np.roll(img_array, i - 1, axis=0)
        for x in range(3):
            temp = np.copy(img_array)
            temp = np.roll(temp, x - 1, axis=1)*kernel[i,x]
            array.append(temp)

    array = np.array(array)
    res = np.sum(array, axis=0)
    res = res.astype(np.int64)
    res = np.clip(res,0,255)
    return res


In [10]:
# Khung là hình tròn.
def square_frame(np_img):
    lx, ly, _ = np_img.shape
    min_value = min(lx, ly)
    X, Y = np.ogrid[0:lx, 0:ly]
    
    mask = (X - lx / 2) ** 2 + (Y - ly / 2) ** 2 > min_value * min_value / 4
    
    np_img[mask,:] = 0
    return np_img


In [11]:
#Khung hình elip
def elip_frame(np_img):
    lx = np_img.shape[0]
    ly = np_img.shape[1]
    
    min_len = min(lx, ly)
    
    axis_1 = 0.37*min_len
    axis_2 = 0.605*min_len
    
    h = ly/2
    k = lx/2
    
    sin_pi4 = np.sin(np.pi/4)
    cos_pi4 = np.cos(np.pi/4)
    
    sub_sinp_pi4 = np.sin(-np.pi/4)
    sub_cos_pi4 = np.cos(-np.pi/4)
    
    for i in range (lx):
        elips = [0,0]
        for j in range (ly):
            elips[0] = 1 if (((j - h) * cos_pi4 + (i - k)*sin_pi4)**2)/axis_1**2 + (((j - h) * sin_pi4 - (i - k)*cos_pi4)**2)/axis_2**2 - 1 > 0 else 0
            elips[1] = 1 if (((j - h) * sub_cos_pi4 + (i - k)*sub_sinp_pi4)**2)/axis_1**2 + (((j - h) * sub_sinp_pi4 - (i - k)*sub_cos_pi4)**2)/axis_2**2 - 1 > 0 else 0
            if(0 in elips): continue
            else: np_img[i][j] = 0 
    return np_img

In [12]:
def readImage(file_name):
        img = Image.open('./' + file_name).convert("RGB")
        np_img = np.asarray(img, dtype='int64')
        return np_img

In [13]:
def main():
    print("0. Thực hiện tất cả chức năng.")
    print("1. Thay đổi độ sáng cho ảnh.")
    print("2. Thay đổi độ tương phản.")
    print("3. Lật ảnh ngang dọc.")
    print("4. Chuyển đổi ảnh RGB thành ảnh xám.")
    print("5. Chồng 2 ảnh cùng kích thước (chỉ làm trên ảnh xám).")
    print("6. Làm mờ ảnh.")
    print("7. Tạo khung hình tròn.")
    print("8. Tạo khung hình elip.")
    print("====================================MENU====================================")
    
    choice = int(input("Chọn chức năng muốn xử lý ảnh: "))
        
    if (choice == 1):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        brightness_value = int(input("Nhập thông số độ sáng cho ảnh: "))
        c1 = change_brightness(np_img, brightness_value)
        
        #Show ảnh
        ListImage = []
        ListImage.append(np_img.copy())
        ListImage.append(c1.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Image Brightness')
            axis[i].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout();
        save(c1, file_name[:-4] + '_brightness.png')

        
    elif (choice == 2):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        contrast_value = int(input("Nhập độ tương phản: "))
        c2 = change_contrast(np_img, contrast_value)
        
        #Show ảnh
        ListImage = []
        ListImage.append(np_img.copy())
        ListImage.append(c2.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Image Contrast')
            axis[i].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout();
        save(c2, file_name[:-4] + '_contrast.png')
        
    elif (choice == 3):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        
        print("a. Lật ảnh theo chiều ngang.")
        print("b. Lật ảnh theo chiều dọc.")
        flip_choice = input("Chọn chiều lật ảnh: ") # Nhấn a để lật ảnh theo chiều ngang, b để lật ảnh theo chiều dọc

        if (flip_choice == "a"):
            c3a = flip_horizontally(np_img)  # Lật ảnh theo chiều ngang
            
            #Show ảnh
            ListImage = []
            ListImage.append(np_img.copy())
            ListImage.append(c3a.copy())
            demo, axis = plt.subplots(1, 2, figsize = (12, 7))
            j = 0
            for i in range(2):
                if  (j == 0): axis[i].set_title(f'Root Image')
                elif(j == 1): axis[i].set_title(f'Image Horizontal')
                axis[i].imshow(ListImage[j], cmap = 'gray')
                j+=1
            plt.tight_layout();
            save(c3a, file_name[:-4] + '_horizontal.png')

        elif (flip_choice == "b"):
            c3b = flip_vertically(np_img)  # Lật ảnh theo chiều dọc
            
            #Show ảnh
            ListImage = []
            ListImage.append(np_img.copy())
            ListImage.append(c3b.copy())
            demo, axis = plt.subplots(1, 2, figsize = (12, 7))
            j = 0
            for i in range(2):
                if  (j == 0): axis[i].set_title(f'Root Image')
                elif(j == 1): axis[i].set_title(f'Image Vertical')
                axis[i].imshow(ListImage[j], cmap = 'gray')
                j+=1
            plt.tight_layout();
            save(c3b, file_name[:-4] + '_vertical.png')
            
    elif (choice == 4):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        c4 = gray_image(np_img)
        
        #Show ảnh
        ListImage = []
        ListImage.append(np_img.copy())
        ListImage.append(c4.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Image Gray')
            axis[i].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout();
        save(c4, file_name[:-4] + '_gray.png')
        
    elif (choice == 5):
        file_name = input("Nhập tên file ảnh thứ nhất: ")
        np_img1 = readImage(file_name)
        file_name = input("Nhập tên file ảnh thứ hai: ")
        np_img2 = readImage(file_name)
        c5 = blending_image(np_img1, np_img2)
        
        #Show ảnh
        ListImage = []
        ListImage.append(np_img1.copy())
        ListImage.append(np_img2.copy())
        if len(c5) != 0:
            j = 0
            demo, axis = plt.subplots(1, 2, figsize = (12, 7))
            for i in range(2):
                if  (j == 0): axis[i].set_title(f'Root Image 1')
                elif(j == 1): axis[i].set_title(f'Root Image 2')
                axis[i].imshow(ListImage[j], cmap = 'gray')
                j+=1
            demo, axis = plt.subplots(1, 1, figsize = (7, 4))
            ListImage.append(c5.copy())
            axis.set_title(f'Image Blend')
            axis.imshow(ListImage[2], cmap = 'gray')
            plt.tight_layout();
            save(c5, file_name[:-4] + '_blend.png')
        else: 
            print("Warning: ")
            print("The sizes of the two images are not equal")
        
        
    elif (choice == 6):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        c6 = gaussian_blur(np_img)
        
        #Show ảnh
        ListImage = []
        ListImage.append(np_img.copy())
        ListImage.append(c6.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Image Blur')
            axis[i].imshow(ListImage[j])
            j+=1
        plt.tight_layout();
        save(c6, file_name[:-4] + '_gray.png')
        
    elif (choice == 7):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        ListImage = []
        ListImage.append(np_img.copy())
        c7 = square_frame(np_img)
        #Show ảnh
        ListImage.append(c7.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Circle Image')
            axis[i].imshow(ListImage[j])
            j+=1
        plt.tight_layout();
        save(c7, file_name[:-4] + '_square_frame.png')
        
    elif (choice == 8):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        ListImage = []
        ListImage.append(np_img.copy())
        c8 = elip_frame(np_img)
        #Show ảnh
        ListImage.append(c8.copy())
        demo, axis = plt.subplots(1, 2, figsize = (12, 7))
        j = 0
        for i in range(2):
            if  (j == 0): axis[i].set_title(f'Root Image')
            elif(j == 1): axis[i].set_title(f'Elip Image')
            axis[i].imshow(ListImage[j])
            j+=1
        plt.tight_layout();
        save(c8, file_name[:-4] + '_elip.png')
        
    elif (choice == 0):
        file_name = input("Nhập tên file ảnh muốn mở: ")
        np_img = readImage(file_name)
        img_copy = []
        img_copy.append(np_img.copy())
        ListImage = []
        ListImage.append(np_img.copy())
        
        #1
        brightness_value = int(input("Nhập thông số độ sáng cho ảnh: "))
        c1 = change_brightness(np_img, brightness_value)
        ListImage.append(c1.copy())
        save(c1, file_name[:-4] + '_brightness.png')
        print("Process brightness success!")
        
        #2
        contrast_value = int(input("Nhập độ tương phản: "))
        c2 = change_contrast(np_img, contrast_value)
        ListImage.append(c2.copy())
        save(c2, file_name[:-4] + '_contrast.png')
        print("Process contrast success!")
        
        #3
        # Lật ảnh theo chiều ngang
        c3a = flip_horizontally(np_img)  
        ListImage.append(c3a.copy())
        save(c3a, file_name[:-4] + '_horizontal.png')
        print("Process flip horizontally image success!")
        # Lật ảnh theo chiều dọc
        c3b = flip_vertically(np_img)
        ListImage.append(c3b.copy())
        save(c3b, file_name[:-4] + '_vertical.png')
        print("Process flip vertically image success!")
        
        #4
        c4 = gray_image(np_img)
        ListImage.append(c4.copy())
        save(c4, file_name[:-4] + '_gray.png')
        print("Process gray image success!")
        
        #5
        print("Chồng hai ảnh.")
        print("a. Sử dụng hai ảnh mới.") #Mở 2 ảnh mới để thực hiện chồng ảnh.
        print("b. Sử dụng ảnh đã mở ban đầu và một ảnh mới.") 
        choice_method = input("Chọn phương thức chồng ảnh: ")
        if(choice_method == "a"):
            file_name1 = input("Nhập tên file ảnh thứ nhất: ")
            np_img1 = readImage(file_name1)
            file_name2 = input("Nhập tên file ảnh thứ hai: ")
            np_img2 = readImage(file_name2)
            c5 = blending_image(np_img1, np_img2)
            if len(c5) != 0:
                ListImage.append(c5.copy())
                save(c5, file_name2[:-4] + '_blend.png')
                print("Process blend two image success!")
           
            else: 
                print("Warning: ")
                ListImage.append(np_img1.copy())
                print("The sizes of the two images are not equal")
                print("Show the replacement image with the first image.")
            
        if(choice_method == "b"):
            file_name2 = input("Nhập tên file ảnh thứ hai: ")
            np_img1 = readImage(file_name2)
            c5 = blending_image(np_img, np_img1)
            if len(c5) != 0:
                ListImage.append(c5.copy())
                save(c5, file_name2[:-4] + '_blend.png')
                print("Process blend two image success!")
            else: 
                ListImage.append(np_img.copy())
                print("Warning: ")
                print("The sizes of the two images are not equal")
                print("Show the replacement image with the first image.")

        #6
        c6 = gaussian_blur(np_img)
        ListImage.append(c6.copy())
        save(c6, file_name[:-4] + '_gray.png')
        print("Process blur success!")
        
        #7
        c7 = square_frame(np_img)
        ListImage.append(c7.copy())
        save(c7, file_name[:-4] + '_square_frame.png')
        print("Process square frame success!")
        
        #8
        c8 = elip_frame(img_copy[0])
        ListImage.append(c8.copy())
        save(c8, file_name[:-4] + '_elip.png')
        print("Processing of two crossed ellipses frames successfully!!")
        
        #Show ảnh
        demo, axis = plt.subplots(1, 1, figsize = (7, 4))
        axis.imshow(ListImage[0])
        axis.set_title('Root Image')
        plt.tight_layout();
        
        demo, axis = plt.subplots(1, 4, figsize = (13, 8))
        j = 1
        for i in range(4):
            if  (j == 1): axis[i].set_title(f'Change Brightness')
            elif(j == 2): axis[i].set_title(f'Change Contrast')
            elif(j == 3): axis[i].set_title(f'Flip Horizontal')
            elif(j == 4): axis[i].set_title(f'Flip Vertical')
            axis[i].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout();  
            
        demo, axis = plt.subplots(1, 3, figsize = (13, 8))
        for it in range(3):
            if  (j == 5): axis[it].set_title(f'Convert Gray')
            elif(j == 6): 
                if len(c5) != 0:
                    axis[it].set_title(f'Blend')
                else:
                    axis[it].set_title(f'Blend - Replacement image')
                    
            elif(j == 7): axis[it].set_title(f'Blur Gausses')
            axis[it].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout();  
        
        demo, axis = plt.subplots(1, 2, figsize = (13, 8))
        for it in range(2):
            if  (j == 8): axis[it].set_title(f'Circle Frame')
            elif(j == 9): axis[it].set_title(f'Elip Frame')
            axis[it].imshow(ListImage[j], cmap = 'gray')
            j+=1
        plt.tight_layout(); 
        
        
        
   

In [ ]:
main()

0. Thực hiện tất cả chức năng.
1. Thay đổi độ sáng cho ảnh.
2. Thay đổi độ tương phản.
3. Lật ảnh ngang dọc.
4. Chuyển đổi ảnh RGB thành ảnh xám.
5. Chồng 2 ảnh cùng kích thước (chỉ làm trên ảnh xám).
6. Làm mờ ảnh.
7. Tạo khung hình tròn.
8. Tạo khung hình elip.
====================================MENU====================================
